# Исследование надежности заемщиков.
Задачи исследования:

1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

3: В основном доход указывают от 50.000 - 1.000.000,однозначно те кто указывают доход менее 200.000 чаще просрачивают платежи.

4: Покупатели недвижимости в кредит на первом месте по своевременным платежам,на втором те кто решили сыграть свадьбу. Те кто покупает авто или идут учиться менее надежны.

### Шаг 1. Обзор данных

In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')
display(df)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-info" role="alert">
Сразу заметны проблемы в данных таблицы:
минусовые значения в стаже,
    в некоторых случаях люди столько не живут в образовании,
разный регистр,
неявные дубликаты в целях
</div>

### Шаг 2.1 Заполнение пропусков

In [27]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [28]:
 part_incom_missed = df['total_income'].isna().sum() / len(df)

<div class="alert alert-info" role="alert">
Найдены случаи пропуска данных в указаных доходах
Их доля составила: 0.10
</div>

In [29]:
median_income = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(median_income)
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


<div class="alert alert-info" role="alert">
Возможно люди не сильно охотно указывают доход и место работы. 
Нас интересует показатель на каждый случай,следовательно данные заполнили медианным значением,к тому же таким образом отдельные 'чемпионы' показателей сильно не повлияют на результат.
</div>

### Шаг 2.2 Проверка данных на аномалии и исправления.

Берем названия столбцов

In [30]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


<div class="alert alert-info" role="alert">
Смотрим что представляют из себя значения столбцов и находим ошибки с разным регистром и отрицательными значениями.там где их не должно быть
</div>

In [31]:
columns = ['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose']
for column in columns:
    print()
    print('Уникальные значения в столбце',column,':')
    print(df[column].unique())
    print(100 * '-')
    



Уникальные значения в столбце children :
[ 1  0  3  2 -1  4 20  5]
----------------------------------------------------------------------------------------------------

Уникальные значения в столбце days_employed :
[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
----------------------------------------------------------------------------------------------------

Уникальные значения в столбце dob_years :
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
----------------------------------------------------------------------------------------------------

Уникальные значения в столбце education :
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
-------


<div class="alert alert-info" role="alert">
Переводим отрицательное значение в положительное
</div>

In [32]:
df['children'] = df['children'].abs()
df['days_employed'] = df['days_employed'].abs()
display(df['children'].unique())


array([ 1,  0,  3,  2,  4, 20,  5])


<div class="alert alert-info" role="alert">
Избавляемся разности регистра в значениях,проверяем результат.
</div>

In [33]:
df['education'] = df['education'].str.lower()
display(df['education'].unique())
df['family_status'] = df['family_status'].str.lower()# Глаз режет,вот и убрал
display(df['family_status'].unique())

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)


<div class="alert alert-info" role="alert">
Заполняем медианой столбец с данными о стаже
</div>

In [34]:
median_days_employed = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(median_income)

### Шаг 2.3. Изменение типов данных.

In [35]:
df['total_income'] = df['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.


<div class="alert alert-info" role="alert">
Удаляем оставшиеся дубликаты и проверяем
</div>

In [36]:
df = df.drop_duplicates().reset_index(drop=True)
display(df.duplicated().sum())

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [37]:
education_dict = df[['education','education_id']].copy()
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
famaly_dict = df[['family_status','family_status_id']].copy()
famaly_dict = famaly_dict.drop_duplicates().reset_index(drop=True)
display(education_dict)
display(famaly_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [38]:
df = df.drop(['family_status'], axis=1)
df = df.drop(['education'], axis=1)
display(df)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

In [39]:
def income_group(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    if income > 1000000:
        return 'A'
df['total_income_category'] = df['total_income'].apply(income_group)
display(df)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

In [40]:
#df.loc[df['purpose'].str.contains('авто'), 'purpose_category'] = 'операции с автомобилем'
def purpose_category(el):
    search = ['авто', 'жи', 'свадьб', 'образован']
    purpose_name = ['операции с автомобилем','операции с недвижимостью','проведение свадьбы','получение образования']
    for i in range(len(search)):
        df.loc[df['purpose'].str.contains(search[i]), 'purpose_category'] = purpose_name[i]
    return df
df = purpose_category(df)
display(df)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


In [41]:
df.groupby('children')['debt'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
children,,
20,76,0.105263
4,41,0.097561
2,2052,0.094542
1,4855,0.091658
3,330,0.081818
0,14091,0.075438
5,9,0.000000


Результат исследование показал: если дети есть то выше вероятность что будет задолжность,но в данном случае связи с количеством никакой нет,если конечно их не 20...

In [42]:
df.groupby('family_status_id')['debt'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
family_status_id,,
4,2810,0.097509
1,4151,0.093471
0,12339,0.075452
3,1195,0.071130
2,959,0.065693


Лидеры по задолжностям те кто живут в гражданском браке,на шаг за ними не женатые.
Меньший сектор риска это разведенные и вдовы.

In [43]:
df.groupby('total_income_category')['debt'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
total_income_category,,
E,22,0.090909
C,16016,0.084915
A,25,0.080000
B,5041,0.070621
D,350,0.060000


В основном доход указывают от 50.000 - 1.000.000,однозначно те кто указывают доход менее 200.000 чаще просрачивают платежи.


In [44]:
df.groupby('purpose_category')['debt'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
purpose_category,,
операции с автомобилем,4306,0.093590
получение образования,4013,0.092200
проведение свадьбы,2324,0.080034
операции с недвижимостью,10811,0.072334


Покупатели недвижимости в кредит на первом месте по своевременным платежам,на втором те кто решили сыграть свадьбу.
Те кто покупает авто или идут учиться менее надежны.

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?



<div class="alert alert-info" role="alert">
Вывод 1: Результат исследование показал: если дети есть то выше вероятность что будет задолжность,но в данном случае связи с количеством никакой нет,если конечно их не 20...
</div>

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?


<div class="alert alert-info" role="alert">
 Вывод 2: Лидеры по задолжностям те кто живут в гражданском браке,на шаг за ними не женатые. Меньший сектор риска это разведенные и вдовы.
</div>

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?



<div class="alert alert-info" role="alert">
Вывод 3: В основном доход указывают от 50.000 - 1.000.000,однозначно те кто указывают доход менее 200.000 чаще просрачивают платежи.
</div>

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?


<div class="alert alert-info" role="alert">
Вывод 4: Покупатели недвижимости в кредит на первом месте по своевременным платежам,на втором те кто решили сыграть свадьбу. Те кто покупает авто или идут учиться менее надежны.
</div>

## Общий вывод:


<div class="alert alert-info" role="alert">
Самый надежный сектор,те кто указывает доход от 200.000,желает жениться\женился и покупает недвижимость.А кто указывает доход менее 200.000 и цель кредита на операции с авто,либо обучением чаще просрачивают платежи.
</div>